In [1]:
# from scipy.optimize import NonlinearConstraint as NonlinCons

In [2]:
# hessian=lambda zeta: (G.T/(mP+G.dot(zeta))**2).dot(G)/G.shape[1]
# cons=NonlinCons(lambda zeta: mP+G.dot(zeta),0,np.inf,jac=lambda zeta: G)
#             res=minimize(target,zeta0,method='trust-constr',jac=gradient,hess=hessian,\
#                          constraints=cons,options={'xtol':1e-5,'gtol':1e-5,'maxiter':10000,'verbose':2})

In [3]:
import cvxpy as cp

In [4]:
zeta=cp.Variable(G.shape[1])
logprob=cp.sum(cp.log(mP+G@zeta))
obj=cp.Maximize(logprob)
cons=[mP+G@zeta>=0]
prob=cp.Problem(obj,cons)
prob.solve(solver=solver,verbose=True,max_iters=500,feastol_inacc=1e-100)
print('Optimization status: {}; final value: {}'.format(prob.status,prob.value))
zeta=zeta.value

In [ ]:
begin=dt.now()
end=dt.now()
print('spent {} seconds'.format((end-begin).seconds))

In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
print(reg.coef_)

[1. 2.]


In [15]:
XX=np.hstack([np.ones([4,1]),X])
beta=np.linalg.solve(XX.T.dot(XX),XX.T.dot(y))
print(beta)

[3. 1. 2.]


In [ ]:
if(all(mP+zeta1.dot(G)>0)):
    zeta=zeta1
else:
    zeta=zeta0

begin=dt.now()
cons=({'type':'ineq','fun':lambda zeta: mP+zeta.dot(G),'jac':lambda zeta: G.T})
res=minimize(target,zeta,method='SLSQP',jac=gradient,constraints=cons,\
             options={'ftol':1e-10,'maxiter':1000})
end=dt.now()
print()
print('Optimization results:')
print('MLE nit: {}, spent {} seconds! '.format(res['nit'],(end-begin).seconds))
if res['success']:
    zeta=res['x']
    print('MLE(Opt) zeta: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f}, norm {:.4f})'\
          .format(zeta.min(),zeta.mean(),zeta.max(),zeta.std(),np.sqrt(np.sum(zeta**2))))
    print('Dist(zeta(Opt),zeta(The))={:.4f}'.format(np.sqrt(np.sum((zeta-zeta1)**2))))
    grad=gradient(zeta)
    print('optimal: value: {:.4f}; grad: (min {:.4f}, mean {:.4f}, max {:.4f}, std {:.4f})'\
          .format(target(zeta),grad.min(),grad.mean(),grad.max(),grad.std()))
    W=self.T(self.S)/(mP+zeta.dot(G))
    self.__estimate(W,'MLE(Opt)',asym=False)
else:
    print('MLE fail')